# Image Analysis

VisualFeatures.CAPTION - Generate a human-readable sentence that describes the content of an image.

VisualFeatures.READ -  Also known as Optical Character Recognition (OCR). Extract printed or handwritten text from images. 
Note: For extracting text from PDF, Office, and HTML documents and document images, use the Document Intelligence service with the Read model. 

VisualFeatures.DENSE_CAPTIONS - Dense Captions provides more details by generating one-sentence captions for up to 10 different regions in the image, including one for the whole image.

VisualFeatures.TAGS - Extract content tags for thousands of recognizable objects, living beings, scenery, and actions that appear in images.

VisualFeatures.OBJECTS - Object detection. This is similar to tagging, but focused on detecting physical objects in the image and returning their location.

VisualFeatures.SMART_CROPS - Used to find a representative sub-region of the image for thumbnail generation, with priority given to include faces.

VisualFeatures.PEOPLE - Detect people in the image and return their location.

## Install Library

In [ ]:
%pip install azure-ai-vision-imageanalysis
%pip install matplotlib
%pip install opencv-python-headless

## Helper Functions

In [ ]:
from matplotlib import pyplot as plt
import cv2

# Function to display an image.
def show_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Convert the image from BGR to RGB for displaying with matplotlib,
    # because OpenCV uses BGR by default and matplotlib expects RGB.
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the image with matplotlib.
    plt.imshow(img_rgb)
    plt.axis("off")  # Turn off axis labels.
    plt.show()

In [ ]:
from PIL import Image, ImageDraw, ImageFont

# Function to display an image with bounding boxes based on the analysis result.
def show_image_with_boxes(image_path, result, type):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

     # Define the font and size
    font = ImageFont.truetype("DejaVuSans-Bold.ttf", 40)  # You can adjust the font size as needed

    if type == "objects":
        if result.objects is not None:
            for object in result.objects.list:
                # Draw bounding box
                box = object.bounding_box
                draw.rectangle([(box['x'], box['y']), (box['x'] + box['w'], box['y'] + box['h'])], outline="red", width=2)
                draw.text((box['x'], box['y'] - 10), f"{object.tags[0].name} ({object.tags[0].confidence:.2f})", fill="red", font=font)
    elif type == "people":
        if result.people is not None:
            for person in result.people.list:
                # Draw bounding box
                box = person.bounding_box
                draw.rectangle([(box['x'], box['y']), (box['x'] + box['w'], box['y'] + box['h'])], outline="red", width=2)
                draw.text((box['x'], box['y'] - 10), f"Confidence {person.confidence:.2f}", fill="red", font=font)
    elif type == "text":
        if result.read is not None:
            for line in result.read.blocks[0].lines:
                for word in line.words:
                    # Draw rectangle over the word
                    bounding_box = word.bounding_polygon
                    draw.polygon([(point['x'], point['y']) for point in bounding_box], outline="red" , width=3)

    # Save or display the image with bounding boxes
    image.show()  # To display the image
    # image.save("output_image_with_boxes.jpg")  # To save the image

## Load Azure Configuration

In [ ]:
import os
azure_computer_vision_endpoint = os.environ["AZURE_COMPUTER_VISION_ENDPOINT"]
azure_computer_vision_key = os.environ["AZURE_COMPUTER_VISION_KEY"]

## Show Image and Create Client

https://learn.microsoft.com/en-us/python/api/overview/azure/ai-vision-imageanalysis-readme?view=azure-python

In [ ]:
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

image = "../data/images/analyze_image.jpg"
show_image(image)

# Read the image file as bytes
with open(image, "rb") as image_file:
    image_data = image_file.read()

# Create an Image Analysis client
client = ImageAnalysisClient(
    endpoint=azure_computer_vision_endpoint,
    credential=AzureKeyCredential(azure_computer_vision_key)
)

## Caption, Dense Captions, Tags

Possible list of visual features: https://learn.microsoft.com/en-us/python/api/azure-ai-vision-imageanalysis/azure.ai.vision.imageanalysis.models.visualfeatures?view=azure-python

In [ ]:
# Get the feature for the image. This will be a synchronously (blocking) call.
result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.CAPTION, VisualFeatures.DENSE_CAPTIONS, VisualFeatures.TAGS],
    gender_neutral_caption=True,  # Optional (default is False)
)

print("Caption:")
if result.caption is not None:
    print(f"   '{result.caption.text}', Confidence {result.caption.confidence:.4f}")

print("Dense Captions:")
for item in result.dense_captions['values']:
    print(f"   '{item['text']}', Confidence {item['confidence']:.4f}")

print("Tags:")
for item in result.tags['values']:
    print(f"   '{item['name']}', Confidence {item['confidence']:.4f}")

## Objects

In [ ]:
# Get the feature for the image. This will be a synchronously (blocking) call.
result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.OBJECTS],
    gender_neutral_caption=True,  # Optional (default is False)
)

 # Print Objects analysis results to the console
print(" Objects:")
if result.objects is not None:
    for object in result.objects.list:
        print(f"   '{object.tags[0].name}', {object.bounding_box}, Confidence: {object.tags[0].confidence:.4f}")
print(f" Image height: {result.metadata.height}")
print(f" Image width: {result.metadata.width}")

show_image_with_boxes(image, result, "objects")

## Smart Crops

Thumbnails often need to have a certain aspect ratio, where aspect ratio is defined as the width in pixels divided by the height in pixels. For example, 1.0 for a square image, or 1.77 for a 16:9 widescreen image.

You can optionally request one or more aspect ratios by setting the `smart_crops_aspect_ratios` argument in the call to `analyze`. Supported values are from 0.75 to 1.8 (inclusive).

If you do not set this value, the service will return one result with an aspect ratio it sees fit between 0.5 and 2.0 (inclusive).

In [ ]:
# Do Smart Cropping analysis on an image stream. This will be a synchronously (blocking) call.
result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.SMART_CROPS],
    smart_crops_aspect_ratios=[1.0],  # Optional. Specify one more desired aspect ratios    
)

if result.smart_crops is not None:
    for smart_crop in result.smart_crops.list:
        print(f"Aspect ratio {smart_crop.aspect_ratio}: Smart crop {smart_crop.bounding_box}")
print(f" Image height: {result.metadata.height}")
print(f" Image width: {result.metadata.width}")


## Show Cropped Image

In [ ]:
from PIL import Image

# Load the original image
original_image = Image.open(image)

# Define the bounding box (x, y, w, h)
# you need to adjust the bounding box based on the smart crop result
bounding_box = {'x': 957, 'y': 0, 'w': 2270, 'h': 2271}

# Crop the image using the bounding box
cropped_image = original_image.crop((bounding_box['x'], bounding_box['y'], 
                                     bounding_box['x'] + bounding_box['w'], 
                                     bounding_box['y'] + bounding_box['h']))

# Display the cropped image
cropped_image.show()

## People

In [ ]:
# Find people in an image stream. This will be a synchronously (blocking) call.
result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.PEOPLE]
)

# Print People analysis results to the console
print(" People:")
if result.people is not None:
    for person in result.people.list:
        print(f"   {person.bounding_box}, Confidence {person.confidence:.4f}")
print(f" Image height: {result.metadata.height}")
print(f" Image width: {result.metadata.width}")

show_image_with_boxes(image, result, "people")

## Text

In [ ]:
image = "../data/images/ocr.jpg"
show_image(image)

# Read the image file as bytes
with open(image, "rb") as image_file:
    image_data = image_file.read()


# Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.READ]
)

# Print text (OCR) analysis results to the console
print(" Read:")
if result.read is not None:
    for line in result.read.blocks[0].lines:
        print(f"   Line: '{line.text}', Bounding box {line.bounding_polygon}")
        for word in line.words:
            print(f"     Word: '{word.text}', Bounding polygon {word.bounding_polygon}, Confidence {word.confidence:.4f}")

print(f" Image height: {result.metadata.height}")
print(f" Image width: {result.metadata.width}")

show_image_with_boxes(image, result, "text")